In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden の TFVision による画像分類

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_tfvision_image_classification.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      View on GitHub
    </a>
  </td>
</table>

## 概要

このノートブックでは、Vertex AI Model Garden で [TFVision](https://github.com/tensorflow/models/blob/master/official/vision/MODEL_GARDEN.md) を使用する方法を紹介します。

### 目的

* 新しいモデルのトレーニング
  * 入力データをトレーニングフォーマットに変換する
  * 新しいモデルをトレーニングするための [hyperparameter tuning jobs](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) を作成する
  * 最適なモデルを見つけてエクスポートする

* トレーニング済みモデルのテスト
  * モデルをモデルレジストリにアップロードする
  * アップロードしたモデルをデプロイする
  * 予測を実行する

* リソースのクリーンアップ

### コスト

このチュートリアルでは、Google Cloud の課金対象となるコンポーネントを使用します。

* Vertex AI
* Cloud Storage

[Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) と [Cloud Storage
pricing](https://cloud.google.com/storage/pricing)について学習し、 [Pricing
Calculator](https://cloud.google.com/products/calculator/)
を使用して、予測される使用量に基づいてコストの見積もりを生成してください。

## 始める前に

In [ ]:
# @title Google Cloud プロジェクトのセットアップ

# @markdown 1. プロジェクトで課金が有効になっていることを確認してください。

# @markdown 2. (オプション) 実験出力の保存用に Cloud Storage バケットを作成します。実験環境の BUCKET_URI を設定します。指定された Cloud Storage バケット（`BUCKET_URI`）は、ノートブックが起動されたのと同じリージョンに配置する必要があります。マルチリージョンバケット（例：「us」）は、マルチリージョン範囲でカバーされる単一リージョン（例：「us-central1」）と一致するとは見なされないことに注意してください。設定されていない場合は、代わりに一意の GCS バケットが作成されます。

import base64
import json
import os
import sys
from datetime import datetime
from io import BytesIO
from typing import Dict, List, Union

import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
import yaml
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from PIL import Image

# デフォルトのクラウドプロジェクトIDを取得します。
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# ジョブ起動のデフォルトリージョンを取得します。
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# "us"、"asia"、または "europe" で始まるリージョンのみがサポートされています。
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} はサポートされていません。"us"、"asia"、または "europe" で始まる必要があります。'

# Vertex AI API と Compute Engine API を有効にします (まだ有効になっていない場合)。
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# 実験アーティファクトを保存するための Cloud Storage バケット。
# このノートブックのために一意の GCS バケットが作成されます。独自の GCS バケットを使用する場合は、以下の値を自分で変更してください。
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://"  # @param {type: "string"}

# ユーザーが指定していない場合は、このノートブックのための一意の GCS バケットを作成します。
if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}"
    BUCKET_NAME = BUCKET_URI
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI は `gs://` で始まる必要があります。"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "バケットリージョン %s はノートブックリージョン %s と異なります"
            % (bucket_region, REGION)
        )

print(f"この GCS バケットを使用します: {BUCKET_URI}")

# デフォルトの SERVICE_ACCOUNT を設定します。
SERVICE_ACCOUNT = None
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print("このデフォルトのサービスアカウントを使用します:", SERVICE_ACCOUNT)

# GCS バケットで SERVICE_ACCOUNT に権限を付与します
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
CHECKPOINT_BUCKET = os.path.join(BUCKET_URI, "ckpt")
CONFIG_DIR = os.path.join(BUCKET_URI, "config")

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)


def upload_config_to_gcs(url):
    """設定ファイルをGCSにアップロードします。"""
    filename = os.path.basename(url)
    destination = os.path.join(CONFIG_DIR, filename)
    print("コピー", url, "から", destination)
    ! wget "$url" -O "$filename"
    ! gsutil cp "$filename" "$destination"


# 各種設定ファイルをGCSにアップロード
upload_config_to_gcs(
    "https://raw.githubusercontent.com/tensorflow/models/master/official/vision/configs/experiments/image_classification/imagenet_resnet50_gpu.yaml"
)
upload_config_to_gcs(
    "https://raw.githubusercontent.com/tensorflow/models/master/official/vision/configs/experiments/image_classification/imagenet_resnetrs50_i160_gpu.yaml"
)
upload_config_to_gcs(
    "https://raw.githubusercontent.com/tensorflow/models/master/official/projects/maxvit/configs/experiments/maxvit_base_imagenet_gpu.yaml"
)

# 定数を定義します。
OBJECTIVE = "icn" # 目的

# データコンバーター定数。
DATA_CONVERTER_JOB_PREFIX = "data_converter" # データコンバータジョブのプレフィックス
DATA_CONVERTER_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/data-converter" # データコンバータコンテナ
DATA_CONVERTER_MACHINE_TYPE = "n1-highmem-8" # データコンバータのマシンタイプ

# トレーニング定数。
TRAINING_JOB_PREFIX = "train" # トレーニングジョブのプレフィックス
TRAIN_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/tfvision-oss" # トレーニングコンテナURI
TRAIN_MACHINE_TYPE = "n1-highmem-16" # トレーニングのマシンタイプ
TRAIN_ACCELERATOR_TYPE = "NVIDIA_TESLA_P100" # トレーニングのアクセラレータタイプ
TRAIN_NUM_GPU = 1 # トレーニングのGPU数

# 評価定数。
EVALUATION_METRIC = "accuracy" # 評価指標

# エクスポート定数。
EXPORT_JOB_PREFIX = "export" # エクスポートジョブのプレフィックス
EXPORT_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/tfvision-model-export" # エクスポートコンテナURI
EXPORT_MACHINE_TYPE = "n1-highmem-8" # エクスポートのマシンタイプ

# 予測定数。
# モデルをデプロイできます
# 事前構築済みDockerを使用:
# 最適化されたTensorFlowランタイムDockerを使用:
# このノートブックの例では、最適化されたTensorFlowランタイムDockerを使用しています。
# より高速な予測を得るために、アクセラレータタイプとマシンタイプを調整できます。
PREDICTION_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.2-11:latest" # 予測コンテナURI
SERVING_CONTAINER_ARGS = ["--allow_precompilation", "--allow_compression"] # サービングコンテナ引数
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4" # 予測のアクセラレータタイプ
PREDICTION_MACHINE_TYPE = "n1-standard-4" # 予測のマシンタイプ
UPLOAD_JOB_PREFIX = "upload" # アップロードジョブのプレフィックス
DEPLOY_JOB_PREFIX = "deploy" # デプロイジョブのプレフィックス


# 共通関数を定義します。
def get_job_name_with_datetime(prefix: str):
    """日時を含むジョブ名を取得します。"""
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def predict_custom_trained_model(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
):
    """カスタムトレーニング済みモデルを予測します。"""
    # AI Platform サービスには、リージョンAPIエンドポイントが必要です。
    client_options = {"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    # リクエストの作成と送信に使用されるクライアントを初期化します。
    # このクライアントは一度だけ作成する必要があり、複数のリクエストで再利用できます。
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    return response.predictions, response.deployed_model_id


def load_img(path):
    """画像を読み込みます。"""
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return Image.fromarray(numpy.uint8(img)).convert("RGB")


def display_image(image):
    """画像を表示します。"""
    _ = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)


def get_prediction_instances(test_filepath, new_width=-1):
    """予測インスタンスを取得します。"""
    if new_width <= 0:
        with tf.io.gfile.GFile(test_filepath, "rb") as input_file:
            encoded_string = base64.b64encode(input_file.read()).decode("utf-8")
    else:
        img = load_img(test_filepath)
        width, height = img.size
        print("元の入力画像サイズ: ", width, " , ", height)
        new_height = int(height * new_width / width)
        new_img = img.resize((new_width, new_height))
        print("サイズ変更後の入力画像サイズ: ", new_width, " , ", new_height)
        buffered = BytesIO()
        new_img.save(buffered, format="JPEG")
        encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")

    instances = [
        {
            "encoded_image": {"b64": encoded_string},
        }
    ]
    return instances


def get_label_map(label_map_yaml_filepath):
    """ラベルマップを取得します。"""
    with tf.io.gfile.GFile(label_map_yaml_filepath, "rb") as input_file:
        label_map = yaml.safe_load(input_file.read())
    return label_map


def get_best_trial(model_dir, max_trial_count, evaluation_metric):
    """最適な試行を取得します。"""
    best_trial_dir = ""
    best_trial_evaluation_results = {}
    best_performance = -1

    for i in range(max_trial_count):
        current_trial = i + 1
        current_trial_dir = os.path.join(model_dir, "trial_" + str(current_trial))
        current_trial_best_ckpt_dir = os.path.join(current_trial_dir, "best_ckpt")
        current_trial_best_ckpt_evaluation_filepath = os.path.join(
            current_trial_best_ckpt_dir, "info.json"
        )
        with tf.io.gfile.GFile(current_trial_best_ckpt_evaluation_filepath, "rb") as f:
            eval_metric_results = json.load(f)
            current_performance = eval_metric_results[evaluation_metric]
            if current_performance > best_performance:
                best_performance = current_performance
                best_trial_dir = current_trial_dir
                best_trial_evaluation_results = eval_metric_results
    return best_trial_dir, best_trial_evaluation_results


def upload_checkpoint_to_gcs(checkpoint_url):
    """チェックポイントをGCSにアップロードします。"""
    filename = os.path.basename(checkpoint_url)
    checkpoint_name = filename.replace(".tar.gz", "")
    print("チェックポイントを", checkpoint_url, "からダウンロードし、", CHECKPOINT_BUCKET, "に保存します")
    ! wget $checkpoint_url -O $filename
    ! mkdir -p $checkpoint_name
    ! tar -xvzf $filename -C $checkpoint_name

    # チェックポイントへの相対パスを検索します。
    checkpoint_path = None
    for root, dirs, files in os.walk(checkpoint_name):
        for file in files:
            if file.endswith(".index"):
                checkpoint_path = os.path.join(root, os.path.splitext(file)[0])
                checkpoint_path = os.path.relpath(checkpoint_path, checkpoint_name)
                break

    ! gsutil cp -r $checkpoint_name $CHECKPOINT_BUCKET/
    checkpoint_uri = os.path.join(CHECKPOINT_BUCKET, checkpoint_name, checkpoint_path)
    print("チェックポイントが", checkpoint_uri, "にアップロードされました")
    return checkpoint_uri



## 新しいモデルのトレーニング

このセクションでは、以下の手順でモデルをトレーニングします。
1. 入力データをトレーニングフォーマットに変換します。
2. 新しいモデルをトレーニングするためのハイパーパラメータ調整ジョブを作成します。
3. 最適なモデルを見つけてエクスポートします。

In [ ]:
# @title トレーニング用の入力データを準備する

# @markdown このセクションでは、入力データをトレーニングフォーマットに変換し、指定された分割比率とシャード数でトレーニング/テスト/検証データセットに分割します。

# @markdown [こちら](https://cloud.google.com/vertex-ai/docs/image-data/classification/prepare-data)に記載されている形式でデータを準備し、以下のようにトレーニングフォーマットに変換します。
# @markdown * `input_file_path`: データ準備用の入力ファイルパス。入力ファイルの例: `gs://cloud-samples-data/ai-platform/flowers/flowers.csv`
# @markdown * `input_file_type`: 入力ファイルの種類。「csv」または「jsonl」を指定できます。
# @markdown * `num_classes`: データセット内のクラス数。
# @markdown * `split_ratio`: トレーニング/検証/テストに分割するデータの比率。例: 「0.8,0.1,0.1」。
# @markdown * `num_shard`: トレーニング/検証/テストのシャード数。例: 「10,10,10」。

# このジョブは、指定された分割比率とトレーニング/テスト/検証のシャード数で、入力データをトレーニングフォーマットに変換します。

from google.cloud.aiplatform import hyperparameter_tuning as hpt

data_converter_job_name = get_job_name_with_datetime(
    DATA_CONVERTER_JOB_PREFIX + "_" + OBJECTIVE
)

input_file_path = "gs://cloud-samples-data/ai-platform/flowers/flowers.csv"  # @param {type:"string"} {isTemplate:true}
input_file_type = "csv"  # @param ["csv", "jsonl"]
num_classes = 5  # @param {type:"integer"}
split_ratio = "0.8,0.1,0.1"  # @param {type:"string"}
num_shard = "10,10,10"  # @param {type:"string"}
data_converter_output_dir = os.path.join(BUCKET_URI, data_converter_job_name)

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": DATA_CONVERTER_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": DATA_CONVERTER_CONTAINER,
            "command": [],
            "args": [
                "--input_file_path=%s" % input_file_path,
                "--input_file_type=%s" % input_file_type,
                "--objective=%s" % OBJECTIVE,
                "--num_shard=%s" % num_shard,
                "--split_ratio=%s" % split_ratio,
                "--output_dir=%s" % data_converter_output_dir,
            ],
        },
    }
]

data_converter_custom_job = aiplatform.CustomJob(
    display_name=data_converter_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

data_converter_custom_job.run()

input_train_data_path = os.path.join(data_converter_output_dir, "train.tfrecord*")
input_validation_data_path = os.path.join(data_converter_output_dir, "val.tfrecord*")
label_map_path = os.path.join(data_converter_output_dir, "label_map.yaml")
print("input_train_data_path for training: ", input_train_data_path)
print("input_validation_data_path for training: ", input_validation_data_path)
print("label_map_path for prediction: ", label_map_path)

In [ ]:
# @title ハイパーパラメータ調整を使用した Vertex AI カスタムジョブの作成と実行

# @markdown このセクションでは、Vertex AI SDK を使用して、Vertex AI Model Garden トレーニング Docker でハイパーパラメータ調整ジョブを作成および実行します。

# @markdown 次の実験から1つを選択します。
# @markdown * tfhub/EfficientNetV2: `Efficientnetv2-m`
# @markdown * tfvision/ViT: `ViT-ti16`, `ViT-s16`, `ViT-b16`, `ViT-l16`
# @markdown * Proprietary/MaxViT: `MaxViT`

# 入力トレーニングデータセットと検証データセットは、上記の「トレーニング用入力データの変換」セクションにあります。
# 準備されたデータセットが存在する場合は設定します。
# input_train_data_path = ''
# input_validation_data_path = ''

experiment = "Efficientnetv2-m"  # @param ["Efficientnetv2-m","ViT-ti16","ViT-s16","ViT-b16","ViT-l16", "MaxViT"]

train_job_name = get_job_name_with_datetime(TRAINING_JOB_PREFIX + "_" + OBJECTIVE)
model_dir = os.path.join(BUCKET_URI, train_job_name)

# ここでの引数は主にテスト用です。より良いパフォーマンスを得るためには、更新してください。
common_args = {
    "input_train_data_path": input_train_data_path,  # トレーニングデータのパス
    "input_validation_data_path": input_validation_data_path,  # 検証データのパス
    "objective": OBJECTIVE,  # 目的
    "model_dir": model_dir,  # モデルの保存先ディレクトリ
    "num_classes": num_classes,  # クラス数
    "global_batch_size": 4,  # グローバルバッチサイズ
    "prefetch_buffer_size": 32,  # プリフェッチバッファサイズ
    "train_steps": 2000,  # トレーニングステップ数
    "input_size": "224,224",  # 入力サイズ
}

# 実験ごとの引数。
experiment_container_args_dict = {
    "Efficientnetv2-m": dict(
        common_args,
        **{
            "experiment": "hub_model",  # 実験名
        },
    ),
    "ViT-ti16": dict(
        common_args,
        **{
            "experiment": "deit_imagenet_pretrain",  # 実験名
            "model_name": "vit-ti16",  # モデル名
            "init_checkpoint": "https://storage.googleapis.com/tf_model_garden/vision/vit/vit-deit-imagenet-ti16.tar.gz",  # 初期チェックポイント
            "input_size": "224,224",  # 入力サイズ
        },
    ),
    "ViT-s16": dict(
        common_args,
        **{
            "experiment": "deit_imagenet_pretrain",  # 実験名
            "model_name": "vit-s16",  # モデル名
            "init_checkpoint": "https://storage.googleapis.com/tf_model_garden/vision/vit/vit-deit-imagenet-s16.tar.gz",  # 初期チェックポイント
            "input_size": "224,224",  # 入力サイズ
        },
    ),
    "ViT-b16": dict(
        common_args,
        **{
            "experiment": "deit_imagenet_pretrain",  # 実験名
            "model_name": "vit-b16",  # モデル名
            "init_checkpoint": "https://storage.googleapis.com/tf_model_garden/vision/vit/vit-deit-imagenet-b16.tar.gz",  # 初期チェックポイント
            "input_size": "224,224",  # 入力サイズ
        },
    ),
    "ViT-l16": dict(
        common_args,
        **{
            "experiment": "deit_imagenet_pretrain",  # 実験名
            "model_name": "vit-l16",  # モデル名
            "init_checkpoint": "https://storage.googleapis.com/tf_model_garden/vision/vit/vit-deit-imagenet-l16.tar.gz",  # 初期チェックポイント
            "input_size": "224,224",  # 入力サイズ
        },
    ),
    "MaxViT": dict(
        common_args,
        **{
            "experiment": "maxvit_imagenet",  # 実験名
            "config_file": os.path.join(CONFIG_DIR, "maxvit_base_imagenet_gpu.yaml"),  # 設定ファイル
        },
    ),
}
experiment_container_args = experiment_container_args_dict[experiment]

# 指定されている場合は、チェックポイントを GCS バケットにコピーします。
init_checkpoint = experiment_container_args.get("init_checkpoint")
if init_checkpoint:
    experiment_container_args["init_checkpoint"] = upload_checkpoint_to_gcs(
        init_checkpoint
    )

# MaxViT をサポートするコンテナを使用します
if experiment == "MaxViT":
    TRAIN_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/tfvision-oss-v2"

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAIN_MACHINE_TYPE,  # マシンタイプ
            "accelerator_type": TRAIN_ACCELERATOR_TYPE,  # アクセラレータタイプ
            # 各トレーニングジョブは TRAIN_NUM_GPU GPU を使用します。
            "accelerator_count": TRAIN_NUM_GPU,  # アクセラレータ数
        },
        "replica_count": 1,  # レプリカ数
        "container_spec": {
            "image_uri": TRAIN_CONTAINER_URI,  # コンテナイメージURI
            "args": [
                "--mode=train_and_eval",  # モード
                "--params_override=runtime.num_gpus=%d" % TRAIN_NUM_GPU,  # GPU数
            ]
            + ["--{}={}".format(k, v) for k, v in experiment_container_args.items()],  # 実験引数
        },
    }
]

metric_spec = {"model_performance": "maximize"}  # 指標仕様


LEARNING_RATES = [5e-4, 1e-3] # 学習率
# モデルはそれぞれの学習率で個別にトレーニングされ、最大試行回数は学習率の数です。
MAX_TRIAL_COUNT = len(LEARNING_RATES)  # 最大試行回数
parameter_spec = {
    "learning_rate": hpt.DiscreteParameterSpec(values=LEARNING_RATES, scale="linear"),  # 学習率のパラメータ仕様
}

print(worker_pool_specs, metric_spec, parameter_spec)

# ハイパーパラメータジョブを実行します。
train_custom_job = aiplatform.CustomJob(
    display_name=train_job_name,  # 表示名
    project=PROJECT_ID,  # プロジェクトID
    worker_pool_specs=worker_pool_specs,  # ワーカープール仕様
    staging_bucket=STAGING_BUCKET,  # ステージングバケット
)

train_hpt_job = aiplatform.HyperparameterTuningJob(
    display_name=train_job_name,  # 表示名
    custom_job=train_custom_job,  # カスタムジョブ
    metric_spec=metric_spec,  # 指標仕様
    parameter_spec=parameter_spec,  # パラメータ仕様
    max_trial_count=MAX_TRIAL_COUNT,  # 最大試行回数
    parallel_trial_count=MAX_TRIAL_COUNT,  # 並列試行回数
    project=PROJECT_ID,  # プロジェクトID
    search_algorithm=None,  # 検索アルゴリズム
)

train_hpt_job.run()  # ジョブを実行

print("experiment is: ", experiment)  # 実験名
print("model_dir is: ", model_dir)  # モデルディレクトリ

In [ ]:
# @title 最適なモデルをTF Saved Model形式でエクスポート

# @markdown このセクションでは、最適なモデルをエクスポートします。

# @markdown エクスポートされたモデルは、次のセクション「トレーニング済みモデルのテスト」でオンライン予測に使用できます。

# TFチェックポイントからTF Saved Model形式にモデルをエクスポートします。
# model_dirは上記のセクションからのものです。
best_trial_dir, best_trial_evaluation_results = get_best_trial(
    model_dir, MAX_TRIAL_COUNT, EVALUATION_METRIC
)
print("best_trial_dir: ", best_trial_dir)
print("best_trial_evaluation_results: ", best_trial_evaluation_results)

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": EXPORT_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": EXPORT_CONTAINER_URI,
            "command": [],
            "args": [
                "--objective=%s" % OBJECTIVE,
                "--input_image_size=%s" % experiment_container_args["input_size"],
                "--experiment=%s" % experiment_container_args["experiment"],
                "--config_file=%s/params.yaml" % best_trial_dir,
                "--checkpoint_path=%s/best_ckpt" % best_trial_dir,
                "--export_dir=%s/best_model" % model_dir,
            ],
        },
    }
]

model_export_name = get_job_name_with_datetime(EXPORT_JOB_PREFIX + "_" + OBJECTIVE)
model_export_custom_job = aiplatform.CustomJob(
    display_name=model_export_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

model_export_custom_job.run()

print("最適なモデルは次の場所に保存されます: ", os.path.join(model_dir, "best_model"))



## Test trained models

In [ ]:
# @title モデルのアップロードとデプロイ

# @markdown このセクションでは、オンライン予測のためにモデルレジストリにモデルをアップロードしてデプロイします。この例では、「新しいモデルのトレーニング」セクションからエクスポートされた最適なモデルを使用します。

trained_model_dir = os.path.join(model_dir, "best_model/saved_model")
upload_job_name = get_job_name_with_datetime(UPLOAD_JOB_PREFIX + "_" + OBJECTIVE)

serving_env = {
    "MODEL_ID": "tensorflow-hub-efficientnetv2",
    "DEPLOY_SOURCE": "notebook",
}

model = aiplatform.Model.upload(
    display_name=upload_job_name,  # 表示名
    artifact_uri=trained_model_dir,  # アーティファクトのURI
    serving_container_image_uri=PREDICTION_CONTAINER_URI,  # サービングコンテナイメージのURI
    serving_container_args=SERVING_CONTAINER_ARGS,  # サービングコンテナの引数
    serving_container_environment_variables=serving_env,  # サービングコンテナの環境変数
)

model.wait()

print("アップロードされたモデル名は次のとおりです:", upload_job_name)

deploy_model_name = get_job_name_with_datetime(DEPLOY_JOB_PREFIX + "_" + OBJECTIVE)
print("デプロイされたジョブ名は次のとおりです:", deploy_model_name)

endpoint = model.deploy(
    deployed_model_display_name=deploy_model_name,  # デプロイされたモデルの表示名
    machine_type=PREDICTION_MACHINE_TYPE,  # マシンタイプ
    traffic_split={"0": 100},  # トラフィック分割
    accelerator_type=PREDICTION_ACCELERATOR_TYPE,  # アクセラレータタイプ
    accelerator_count=1,  # アクセラレータの数
    min_replica_count=1,  # 最小レプリカ数
    max_replica_count=1,  # 最大レプリカ数
)

endpoint_id = endpoint.name
print("エンドポイントIDは次のとおりです:", endpoint_id)



In [ ]:
# @title 予測の実行

# @markdown デプロイが成功したら、オンライン予測のためにエンドポイントに画像を送信できます。

# @markdown `test_filepath`：テスト画像ファイルへのGCS URI。URIは "gs://" で始まる必要があります。

# endpoint_idは上記のセクション（「モデルのアップロードとデプロイ」）で生成されました。
endpoint_id = endpoint.name

test_filepath = "gs://cloud-samples-data/ai-platform/flowers/roses/9423755543_edb35141a3_n.jpg"  # @param {type:"string"} {isTemplate:true}
# 入力画像が大きすぎる場合は、予測のためにサイズを変更します。
instances = get_prediction_instances(test_filepath, new_width=1000)

# ラベルマップファイルは上記のセクション（「トレーニング用の入力データの変換」）で生成されました。
label_map = get_label_map(label_map_path)["label_map"]

predictions, _ = predict_custom_trained_model(
    project=PROJECT_ID, location=REGION, endpoint_id=endpoint_id, instances=instances
)

probs = dict(predictions[0])["probs"]
max_prob = max(probs)
max_index = probs.index(max_prob)
print("テスト画像: ", test_filepath)
print("最大確率: ", max_prob, ", ラベル: ", label_map[max_index])
img = load_img(test_filepath)
display_image(img)


## Clean up

In [ ]:
# @title リソースのクリーンアップ

# @markdown 実験で使用したモデルとエンドポイントを削除して、リソースをリサイクルし、
# @markdown 不要な継続課金を回避します。

try:
    # モデルをアンデプロイし、エンドポイントを削除します。
    endpoint.delete(force=True)

    # モデルを削除します。
    model.delete()

except Exception as e:
    print(e)

try:
    # カスタムジョブと HPT ジョブを削除します。
    if data_converter_custom_job.list(
        filter=f'display_name="{data_converter_job_name}"'
    ):
        data_converter_custom_job.delete()
    if train_hpt_job.list(filter=f'display_name="{train_job_name}"'):
        train_hpt_job.delete()
    if model_export_custom_job.list(filter=f'display_name="{model_export_name}"'):
        model_export_custom_job.delete()
except Exception as e:
    print(e)

# バケットを削除します。
delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME

